# STAC Catalog 

Invoke a Common Workflow Language CommandLineTool to generate the STAC Catalog

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/stac



## Setup

In [1]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.3.0/app-water-bodies.0.3.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the STAC Catalog generation CommandLineTool

Inspect and use `cwltool` to run the STAC Catalog generation definition:


In [2]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][6]' -

class: CommandLineTool
id: stac
requirements:
  InlineJavascriptRequirement: {}
  EnvVarRequirement:
    envDef:
      PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
      PYTHONPATH: /app
  ResourceRequirement:
    coresMax: 1
    ramMax: 512
hints:
  DockerRequirement:
    dockerPull: ghcr.io/eoap/mastering-app-package/stac@sha256:cb54ab738a7d6544f2037368604c1c40e3f5af3eac1250ed14d365e5acb0c6b5
baseCommand: ["python", "-m", "app"]
arguments: []
inputs:
  item:
    type:
      type: array
      items: string
      inputBinding:
        prefix: --input-item
  rasters:
    type:
      type: array
      items: File
      inputBinding:
        prefix: --water-body
outputs:
  temp_stac_catalog:
    label: temporary STAC catalog with COG outputs
    outputBinding:
      glob: .
    type: Directory


Run the CWL description, but first prepare the parameters.

The previous step generated the water bodies detection geotif:

In [3]:
cat water-bodies-results.json 

{
    "detected_water_body": {
        "location": "file:///workspace/zarr-cloud-native-format/runs/otsu.tif",
        "basename": "otsu.tif",
        "class": "File",
        "checksum": "sha1$a7f9a22f096d7bb5e2a0ecfe0abba2eab2350f1f",
        "size": 1100785,
        "path": "/workspace/zarr-cloud-native-format/runs/otsu.tif"
    }
}


Let's build the job parameters file with the otsu.tif file and the associated STAC Item input

In [10]:
ostu_tif=$(cat water-bodies-results.json  | jq '.detected_water_body.path')

echo ${ostu_tif}

"/workspace/zarr-cloud-native-format/runs/otsu.tif"


In [9]:
item=$( cat convert-search-results.json | jq '.items[0]' )

echo ${item}

"https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A"


In [14]:
cat <<EOF > stac-generation-params.yaml
item: 
- ${item}
rasters:
- class: File
  path: ${ostu_tif}
EOF

cat stac-generation-params.yaml | yq .

item:
  - "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A"
rasters:
  - class: File
    path: "/workspace/zarr-cloud-native-format/runs/otsu.tif"


In [15]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#stac \
    stac-generation-params.yaml > stac-generation-results.json 2> stac-generation.log

Let's look at the content of the stderr:

In [16]:
cat stac-generation.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac'
INFO [job stac] /tmp/giqo2xko$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/giqo2xko,target=/RhFOvY \
    --mount=type=bind,source=/tmp/p0mjuywr,target=/tmp \
    --mount=type=bind,source=/workspace/zarr-cloud-native-format/runs/otsu.tif,target=/var/lib/cwl/stg026e359f-2aa4-4147-8064-d8afa89091ca/otsu.tif,readonly \
    --workdir=/RhFOvY \
    --read-only=true \
    --user=1000:1000 \
    --rm \
    --cidfile=/tmp/7jv6tx7d/20250915144715-676009.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/RhFOvY \
    --env=PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin \
    --env=PYTHONPATH=/app \
    ghcr.io/eoap/mastering-app-package/stac@sha256:cb54ab738a7d6544f2037368604c1c40e3f5af3eac1250ed14d365e5acb0c6b5 

Let's inspect the stdout produced. There are two `output` blocks with the id `aoi` and `items`. 

These are what the water bodies detection sub-workflow expects as inputs

In [17]:
cat stac-generation-results.json | jq . -

{
  "temp_stac_catalog": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko",
    "basename": "giqo2xko",
    "class": "Directory",
    "listing": [
      {
        "class": "File",
        "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko/catalog.json",
        "basename": "catalog.json",
        "size": 363,
        "checksum": "sha1$60975eb0efd2cb8615bfa4b181774ea961ee7d54",
        "path": "/workspace/zarr-cloud-native-format/runs/giqo2xko/catalog.json"
      },
      {
        "class": "Directory",
        "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko/S2A_10TFK_20210728_0_L2A",
        "basename": "S2A_10TFK_20210728_0_L2A",
        "listing": [
          {
            "class": "File",
            "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko/S2A_10TFK_20210728_0_L2A/S2A_10TFK_20210728_0_L2A.json",
            "basename": "S2A_10TFK_20210728_0_L2A.json",
            "size": 2774,
         